# Documentation Technique - Decision Service

## Vue d'ensemble du projet PIE Eye-Tracking

Le **Decision Service** est un microservice FastAPI qui gère la logique de décision basée sur le suivi du regard (eye-tracking) pour contrôler une interface domotique intelligente.

### Architecture Générale

```
┌─────────────────┐
│  Video Service  │ ──► Flux vidéo en temps réel
└─────────────────┘
         │
         ▼
┌──────────────────┐
│ OpenFace Service │ ──► Analyse faciale + extraction du regard (gaze)
└──────────────────┘
         │
         ▼ (gRPC/ZMQ)
┌──────────────────┐
│ Decision Service │ ──► Logique de décision + Interface utilisateur
└──────────────────┘
         │
         ▼ (WebSocket)
┌──────────────────┐
│   Client Web     │ ──► Affichage de l'interface domotique
└──────────────────┘
```

### Composants Principaux

1. **Routers** : Gestion des endpoints HTTP et WebSocket
2. **Services** : Logique métier (gaze processing, connection manager, IHM)
3. **Core** : États partagés et gestion de la concurrence
4. **Templates** : Interfaces HTML pour l'affichage

## 1. Configuration de l'Environnement

### Dépendances Requises

Le service utilise Python avec les bibliothèques suivantes :

- **FastAPI (0.121.3)** : Framework web asynchrone moderne pour créer des APIs
- **uvicorn (0.38.0)** : Serveur ASGI pour exécuter FastAPI
- **websockets (15.0.1)** : Communication en temps réel bidirectionnelle
- **grpcio (1.70.0)** : Communication avec le service OpenFace via gRPC
- **opencv-python (4.12.0.88)** : Traitement et manipulation d'images
- **pyzmq (27.1.0)** : Messaging pattern (alternative à gRPC pour certains flux)
- **pydantic (2.12.4)** : Validation et sérialisation de données

### Installation

```bash
cd decision-service
pip install -r requirements.txt
```

### Lancement du service

```bash
uvicorn app.main:app --reload --host 0.0.0.0 --port 8000
```

## 2. Structure du Projet

```
decision-service/
├── app/
│   ├── main.py                    # Point d'entrée FastAPI
│   ├── core/
│   │   └── state.py              # États globaux et verrous
│   ├── routers/
│   │   ├── ws.py                 # WebSocket endpoint
│   │   ├── interface.py          # Routes d'interface
│   │   └── configuration.py      # Routes de configuration
│   ├── services/
│   │   ├── connection_manager.py # Gestion des connexions WS
│   │   ├── gaze.py              # Traitement du regard
│   │   ├── logic_wheel.py       # Logique de la roue de décision
│   │   ├── IHM.py               # Gestion de l'interface
│   │   └── process_frame.py     # Traitement des frames
│   ├── templates/               # Templates HTML
│   └── config/                  # Fichiers de configuration JSON
├── requirements.txt
└── Dockerfile
```

## 3. Module Core - Gestion des États

Le module `core/state.py` gère les états globaux de l'application avec des verrous asynchrones pour garantir la cohérence des données.

### États disponibles

**CalibrationStatus**
- `IDLE` : Pas de calibration en cours
- `RUNNING` : Calibration active

**GazeVisualisationStatus**
- `RUNNING` : Visualisation du regard active
- `NOT_RUNNING` : Visualisation arrêtée

**VideoStatus**
- `RUNNING` : Flux vidéo actif
- `NOT_RUNNING` : Flux vidéo arrêté

**VideoServiceConnection**
- `CONNECTED` : Service vidéo connecté
- `DISCONNECTED` : Service vidéo déconnecté

### Utilisation des verrous

Chaque état est protégé par un `asyncio.Lock()` pour éviter les race conditions lors des accès concurrents :

```python
async with state.gaze_visualisation_lock:
    state.gaze_visualisation_status = state.GazeVisualisationStatus.RUNNING
```

## 4. Connection Manager - Gestion des WebSockets

Le `ConnectionManager` gère toutes les connexions WebSocket actives et permet le broadcast de messages.

### Architecture

Le manager maintient une liste de connexions actives, chacune identifiée par son **type de client** :
- `video` : Pour le flux vidéo
- `interface` : Pour l'interface utilisateur
- `gaze` : Pour les données de regard
- `calibration` : Pour la calibration

### Méthodes principales

**`connect(websocket, client_type)`**
- Accepte une nouvelle connexion WebSocket
- L'enregistre avec son type dans la liste active
- Affiche le nombre total de connexions

**`disconnect(websocket_or_conn)`**
- Déconnecte un client proprement
- Accepte soit l'objet WebSocket, soit le dictionnaire de connexion
- Nettoie automatiquement la liste des connexions

**`broadcast(data, client_type)`**
- Envoie des données à tous les clients d'un type spécifique
- Pour le type `video` : envoie en **bytes** (frames encodées)
- Pour les autres types : envoie en **JSON**
- Détecte et nettoie automatiquement les connexions mortes

### Gestion des erreurs

Le broadcast gère trois types d'exceptions :
- `WebSocketDisconnect` : Déconnexion normale
- `RuntimeError` : Erreur d'état du WebSocket
- `ConnectionResetError` : Connexion fermée brutalement

## 5. La Roue de Décision (DecisionWheel)

Le cœur du système de décision. Utilise un mécanisme de **résilience totale** avec buffer pour éviter les validations accidentelles.

### Principe de fonctionnement

1. L'utilisateur regarde une direction (UP, DOWN, LEFT, RIGHT, CENTER)
2. Un compteur se remplit progressivement (ex: 4 ticks pour atteindre 100%)
3. Si l'utilisateur détourne le regard, un **buffer de tolérance** se déclenche
4. Le buffer protège le progrès pendant un certain nombre de ticks
5. Si le regard revient, le compteur continue
6. Si le buffer est dépassé, le progrès est perdu

### Implémentation de la DecisionWheel

**Paramètres de configuration**
- `seuil_validation` : Nombre de ticks nécessaires pour valider (défaut: 4)
- `buffer_limit` : Nombre de ticks de tolérance (défaut: 2)

**Attributs internes**
- `compteur` : Progression actuelle vers la validation (0 à seuil)
- `buffer_compteur` : Compteur de tolérance actif
- `direction_en_cours` : Direction actuellement trackée
- `decision_validee` : Direction validée à 100% (ou None)

**Méthode `update(raw_input_direction)`**

Cette méthode est appelée à chaque frame et retourne un objet contenant :
- `direction` : Direction affichée (peut différer de l'input brut)
- `percent` : Pourcentage de progression (0-100%)
- `validated` : Direction validée si 100% atteint
- `status` : `"RUNNING"` ou `"PAUSE"` (si buffer actif)

### Logique de résilience

**Cas 1 : Continuation dans la même direction**
- Reset du buffer (pas de stress)
- Incrémentation du compteur
- Validation si seuil atteint

**Cas 2 : Changement de direction**
- Si compteur > 0 : **activation du buffer**
- Pendant buffer_limit ticks : **on ignore le changement**
- Si buffer dépassé : réinitialisation et basculement

**Cas 3 : Démarrage depuis CENTER**
- Pas de buffer nécessaire
- Basculement immédiat vers la nouvelle direction
- Compteur démarre à 1 (25%)

## 6. Système Domotique (SmartHomeSystem)

Le `SmartHomeSystem` gère l'état de la maison intelligente avec ses différentes pièces et appareils.

### Structure des données

La maison est organisée par **pièces**, chaque pièce contenant une liste d'**appareils** :

```json
{
  "SALON": {
    "devices": [
      {"id": "light_main", "name": "Lumière Plafond", "type": "binary", "state": false},
      {"id": "shutters", "name": "Volets", "type": "analog", "state": 0},
      {"id": "speaker", "name": "Enceinte", "type": "analog", "state": 30}
    ]
  },
  "CUISINE": {"devices": []},
  "CHAMBRE": {"devices": []},
  "SDB": {"devices": []}
}
```

### Types d'appareils

**Binary (Binaire)**
- État : `true` (allumé) ou `false` (éteint)
- Exemples : Lumières, prises électriques
- Actions : ON/OFF

**Analog (Analogique)**
- État : Valeur de 0 à 100 (%)
- Exemples : Volets, volume des enceintes, intensité lumineuse
- Actions : +10% ou -10% par commande

### Configuration via JSON

**TODO actuel** : La configuration est hardcodée dans `logic_wheel.py`

**Évolution prévue** : Lecture depuis `config/smart_home_config.json` pour permettre :
- Ajout facile de nouvelles pièces
- Modification des appareils sans toucher au code
- Configuration personnalisée par utilisateur

## 7. Gestionnaire d'Interface (InterfaceManager)

L'`InterfaceManager` orchestre l'affichage de l'interface et gère la navigation entre les différents modes.

### Modes de navigation

**MENU_PRINCIPAL**
- Affiche le choix des pièces
- UP → SALON
- LEFT → CUISINE
- RIGHT → CHAMBRE
- DOWN → SDB
- CENTER → "MAISON" (texte informatif)

**ROOM_CONTROL**
- Contrôle des appareils dans une pièce
- UP/DOWN → Actions sur l'appareil sélectionné
- LEFT/RIGHT → Navigation entre appareils (carrousel)
- CENTER → Affichage de l'appareil actuel

### Méthode `get_ui_context()`

Appelée **4 fois par seconde** (à chaque frame), cette méthode prépare toutes les données pour l'affichage :

**Retour de la fonction :**
- `labels` : Textes à afficher dans chaque cercle (UP, DOWN, LEFT, RIGHT, CENTER)
- `room_name` : Nom de la pièce actuelle
- `queue` : Liste des appareils avec leur état
- `center_theme` : Thème de couleur du cercle central

### Gestion intelligente des labels

L'interface s'adapte automatiquement selon le contexte :

**Pour une lumière éteinte :**
- UP → "ALLUMER"
- DOWN → (vide)

**Pour une lumière allumée :**
- UP → (vide)
- DOWN → "ÉTEINDRE"

**Pour un appareil analogique :**
- UP → "+"
- DOWN → "-"

### Thèmes visuels

- `light-on` : Cercle jaune (lumière allumée)
- `light-off` : Cercle gris sombre (lumière éteinte)
- `neutral` : Cercle gris (par défaut)

### Méthode `process_validation(direction)`

Appelée **uniquement quand la roue atteint 100%** (validation complète).

**Dans le MENU_PRINCIPAL :**
- Validation UP → Passage en mode `ROOM_CONTROL` dans le SALON
- Reset de `selected_device_index` à 0

**Dans le ROOM_CONTROL :**
- RIGHT → Appareil suivant (modulo)
- LEFT → Appareil précédent (modulo)
- UP/DOWN → Modification de l'état de l'appareil via `SmartHomeSystem.update_device()`

**Note importante** : Le CENTER ne déclenche jamais d'action de validation.

## 8. Flux de Données Complet

### Étape 1 : Capture vidéo
Le **video-service** capture la webcam et envoie les frames via WebSocket au decision-service.

### Étape 2 : Analyse faciale
Le **openface-service** reçoit les frames et extrait :
- Les landmarks faciaux (68 points)
- Les angles de regard (gaze_angle_x, gaze_angle_y)

Communication via **gRPC** ou **ZMQ** selon la configuration.

### Étape 3 : Calibration
Le module `gaze.py` utilise les données de calibration pour normaliser les coordonnées :
- `gax_min`, `gax_max` : Plage horizontale du regard
- `gay_min`, `gay_max` : Plage verticale du regard

Transformation en coordonnées normalisées (0 à 1).

### Étape 4 : Détection de zone
Le regard normalisé est converti en direction (UP, DOWN, LEFT, RIGHT, CENTER) selon des seuils prédéfinis.

### Étape 5 : Mise à jour de la roue
La `DecisionWheel` reçoit la direction et met à jour son état interne avec la logique de buffer.

### Étape 6 : Préparation du contexte UI
L'`InterfaceManager` prépare les données d'affichage en fonction du mode actuel et de l'état de la maison.

### Étape 7 : Broadcast WebSocket
Le `ConnectionManager` envoie les données au client web :
- État de la roue (direction, pourcentage)
- Contexte UI (labels, pièce, liste d'appareils)
- Frame vidéo annotée (optionnel)

### Étape 8 : Affichage
Le client web (HTML/JavaScript) reçoit les données et met à jour l'interface en temps réel (4 FPS).

## 9. Endpoints API

### WebSocket `/ws`

**Paramètre de connexion :** `type` (query parameter)

Endpoint principal pour la communication temps réel. Maintient la connexion ouverte avec un sleep de 30ms.

```
ws://localhost:8000/ws?type=interface
```

Types de clients supportés :
- `video` : Reçoit les frames vidéo en bytes
- `interface` : Reçoit les mises à jour UI en JSON
- `gaze` : Reçoit les données de regard brutes
- `calibration` : Reçoit les données de calibration

### Routers HTTP (interface.py)

Routes pour servir les templates HTML :
- `/` : Page d'accueil
- `/calibration` : Interface de calibration
- `/gaze_visualisation` : Visualisation du regard en temps réel
- `/IHM` : Interface principale de contrôle domotique
- `/perf` : Monitoring des performances

### Routers de Configuration (configuration.py)

Routes pour contrôler l'état du service :
- `POST /start_calibration` : Démarre la calibration
- `POST /stop_calibration` : Arrête la calibration
- `GET /calibration_status` : État de la calibration
- `POST /start_gaze_visualisation` : Démarre la visualisation
- `POST /stop_gaze_visualisation` : Arrête la visualisation
- `GET /video_service_status` : État de la connexion au service vidéo

## 10. Services Annexes

### process_frame.py

Gère le traitement continu des frames vidéo. Maintient en mémoire :
- `latest_frame` : Dernière frame reçue
- `latest_landmark` : Derniers landmarks détectés
- `latest_gaze` : Dernières données de regard

Tâche asynchrone qui tourne en arrière-plan.

### gaze.py

**Fonctions principales :**

**`gaze_visualisation()`**
- Lance le processus de visualisation du regard
- Utilise les verrous d'état pour la thread-safety
- Gère le cycle de vie (démarrage/arrêt)

**`process_gaze_visualisation()`**
- Boucle infinie qui traite les données de regard
- Normalise les coordonnées avec les données de calibration
- Broadcast aux clients WebSocket de type "gaze"

**Données de calibration :**
```json
{
  "gax_max": 0.0728,
  "gax_min": -0.195,
  "gay_max": 0.177,
  "gay_min": -0.150
}
```

Ces valeurs définissent les limites du regard calibré pour l'utilisateur.

### IHM.py

Intègre les composants :
- `DecisionWheel`
- `SmartHomeSystem`
- `InterfaceManager`

Orchestre la communication entre la logique de décision et l'affichage de l'interface.

## 11. Communication gRPC avec OpenFace

### Protocole

Le service utilise **Protocol Buffers (protobuf)** pour la communication avec OpenFace Service.

**Fichiers générés :**
- `openfaceservice_pb2.py` : Définitions des messages
- `openfaceservice_pb2_grpc.py` : Stubs client/serveur
- `openfaceservice_pb2.pyi` : Types pour l'auto-complétion

### Structure de communication

**Requête → OpenFace Service**
- Frame vidéo (bytes)
- Configuration de traitement

**Réponse ← OpenFace Service**
- Landmarks faciaux (liste de coordonnées)
- Angles de regard (gaze_angle_x, gaze_angle_y)
- Probabilité de détection
- Métadonnées (timestamp, frame_id)

### Alternative ZMQ

Le système peut aussi utiliser **ZeroMQ** (pyzmq) pour :
- Communication publish/subscribe
- Latence plus faible que gRPC pour certains cas
- Simplicité de configuration

**Configuration actuelle :**
```
IP : 172.26.128.105
Port : 8081
```

## 12. Déploiement avec Docker

### Dockerfile

Le service est containerisé avec :
- Base image Python
- Installation des dépendances système (OpenCV, etc.)
- Copie du code applicatif
- Exposition du port 8000
- Script de lancement `launch_app.sh`

### Docker Compose

L'architecture complète utilise `docker-compose.yaml` à la racine du projet pour orchestrer :

1. **video-service** : Capture et streaming vidéo
2. **openface-service** : Analyse faciale et extraction du regard
3. **decision-service** : Logique de décision et interface

**Réseau interne** : Les services communiquent via un réseau Docker privé.

**Volumes** : Partage de données entre conteneurs si nécessaire.

### Commandes

**Lancer tous les services :**
```bash
docker-compose up --build
```

**Lancer uniquement le decision-service :**
```bash
docker-compose up decision-service
```

**Voir les logs :**
```bash
docker-compose logs -f decision-service
```

## 13. Évolutions Futures (TODOs)

### Configuration JSON

**Priorité : Haute**

Actuellement, la configuration de la maison intelligente est hardcodée dans `logic_wheel.py`. Il faut :

1. Créer le fichier `app/config/smart_home_config.json`
2. Charger la configuration au démarrage de `SmartHomeSystem`
3. Permettre le rechargement à chaud (sans redémarrage)

**Avantages :**
- Ajout facile de pièces (cuisine, garage, etc.)
- Modification des appareils sans toucher au code
- Configuration personnalisée par utilisateur
- Support multi-utilisateurs

### Configuration de l'interface

**Priorité : Moyenne**

Externaliser aussi la configuration du menu principal dans un JSON pour :
- Personnaliser les labels des boutons
- Modifier la structure de navigation
- Supporter plusieurs langues

### Amélioration du buffer

**Priorité : Basse**

Rendre les paramètres de la roue configurables :
- Seuil de validation ajustable
- Buffer limit dynamique selon l'utilisateur
- Calibration personnalisée des zones de regard

### Logging et Monitoring

**Priorité : Moyenne**

Ajouter un système de logging complet :
- Traçabilité des actions utilisateur
- Métriques de performance (FPS, latence)
- Détection des anomalies
- Export des données pour analyse

## 14. Performances et Optimisations

### Fréquence de rafraîchissement

**Actuel :** 4 FPS (250ms par frame)
- `get_ui_context()` appelée 4 fois/seconde
- WebSocket sleep de 30ms dans la boucle principale

**Optimisations possibles :**
- Adapter la fréquence selon la charge système
- Réduire la fréquence quand inactif (mode veille)
- Augmenter à 10 FPS pour plus de réactivité

### Gestion mémoire

**Points d'attention :**
- `latest_frame` stocke la dernière frame en mémoire
- Nettoyage automatique des connexions WebSocket mortes
- Pas de fuite mémoire détectée avec les verrous asyncio

### Latence bout-en-bout

**Chaîne complète :**
1. Capture webcam → 30-60ms
2. Analyse OpenFace → 50-100ms
3. Decision Service → 10-20ms
4. Affichage client → 10-20ms

**Total :** ~100-200ms de latence (acceptable pour l'usage)

### Scalabilité

**Limitations actuelles :**
- Un seul utilisateur à la fois
- État global non distribué
- Pas de persistance des données

**Pour supporter plusieurs utilisateurs :**
- Session par utilisateur (cookies/JWT)
- Base de données pour l'état
- Load balancing avec Redis

## 15. Scénario d'Utilisation Complet

### Étape 1 : Démarrage du système

L'utilisateur lance les 3 services via Docker Compose. Le decision-service démarre et expose l'API sur le port 8000.

### Étape 2 : Connexion du client

L'utilisateur accède à `http://localhost:8000/IHM` dans son navigateur. Une connexion WebSocket s'établit automatiquement.

### Étape 3 : Calibration initiale

Avant la première utilisation, l'utilisateur lance la calibration via `/calibration`. Il regarde les 4 coins de l'écran pour définir les limites de son champ de regard.

### Étape 4 : Navigation dans le menu

L'interface affiche le **MENU PRINCIPAL** avec 4 options (SALON, CUISINE, CHAMBRE, SDB). L'utilisateur regarde vers le haut (UP) pour sélectionner le SALON.

### Étape 5 : Remplissage de la roue

La roue de décision se remplit progressivement :
- 25% → 1 tick
- 50% → 2 ticks
- 75% → 3 ticks
- 100% → 4 ticks (VALIDATION)

Si l'utilisateur détourne le regard, le buffer de 2 ticks se déclenche pour protéger le progrès.

### Étape 6 : Entrée dans le SALON

À 100%, le mode bascule en **ROOM_CONTROL**. L'interface affiche les 3 appareils du salon :
1. Lumière Plafond (OFF)
2. Volets (0%)
3. Enceinte (30%)

### Étape 7 : Contrôle de la lumière

L'utilisateur regarde UP pour allumer la lumière. La roue se remplit à nouveau, et à 100%, la lumière s'allume. Le cercle central devient jaune.

### Étape 8 : Navigation vers l'enceinte

L'utilisateur regarde RIGHT deux fois pour sélectionner l'enceinte (appareil 3).

### Étape 9 : Modification du volume

L'utilisateur regarde UP plusieurs fois pour augmenter le volume de 30% à 60% (3 validations × 10%).

## 16. Dépannage et Problèmes Courants

### WebSocket se déconnecte fréquemment

**Cause :** Timeout ou problème réseau

**Solution :**
- Vérifier que le sleep de 30ms dans `/ws` est actif
- Augmenter le timeout du serveur uvicorn
- Vérifier les logs pour des exceptions

### La roue ne se remplit pas

**Causes possibles :**
1. OpenFace Service non connecté → Vérifier `video_service_status`
2. Calibration incorrecte → Refaire la calibration
3. Seuils de détection trop stricts → Ajuster dans le code

**Debug :**
- Ouvrir `/gaze_visualisation` pour voir les données brutes
- Vérifier les coordonnées de regard dans la console

### L'interface ne se met pas à jour

**Cause :** Client WebSocket non connecté

**Solution :**
- F12 → Console pour voir les erreurs JavaScript
- Vérifier que le type de connexion est correct (`?type=interface`)
- Redémarrer le service si nécessaire

### Les appareils ne changent pas d'état

**Cause :** La validation n'est pas déclenchée ou l'action est incorrecte

**Debug :**
- Ajouter des prints dans `process_validation()`
- Vérifier que `decision_validee` n'est pas None
- Confirmer que le mode est bien `ROOM_CONTROL`

### Docker ne démarre pas

**Causes :**
- Port 8000 déjà utilisé → Changer le port dans docker-compose.yaml
- Dépendances manquantes → Rebuilder l'image (`docker-compose build`)
- Problèmes de réseau Docker → Redémarrer Docker Desktop

## 17. Conclusion et Résumé

### Points Clés du Decision Service

1. **Architecture modulaire** : Séparation claire entre routers, services et core
2. **Communication temps réel** : WebSocket pour une latence minimale
3. **Résilience totale** : Système de buffer pour éviter les erreurs de saisie
4. **Adaptabilité** : Interface qui s'adapte au contexte (mode, appareil)
5. **Scalabilité Docker** : Déploiement simplifié avec containers

### Technologies Utilisées

- **FastAPI** : Framework web asynchrone performant
- **WebSocket** : Communication bidirectionnelle en temps réel
- **gRPC/ZMQ** : Communication inter-services
- **OpenCV** : Traitement d'images
- **asyncio** : Gestion de la concurrence

### Prochaines Étapes de Développement

1. ✅ Architecture de base fonctionnelle
2. ✅ Roue de décision avec buffer
3. ✅ Système domotique basique
4. 🔲 Configuration JSON externalisée
5. 🔲 Support multi-utilisateurs
6. 🔲 Persistance des données
7. 🔲 Monitoring et analytics
8. 🔲 Interface mobile/responsive

### Contacts et Ressources

**Projet :** PIE Eye-Tracking - Olympio
**Service :** Decision Service
**Framework :** FastAPI + WebSocket
**Déploiement :** Docker Compose

---

*Documentation générée le 11 février 2026*